In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque
from ultralytics import YOLO

# --- Initialize Mediapipe ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# --- Load YOLOv10n model ---
yolo = YOLO("yolov8n.pt")  # Change to yolov10n.pt when available

# Eye landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

score_history = deque(maxlen=10)
distraction = 0

def eye_aspect_ratio(landmarks, eye_points, image_w, image_h):
    p = []
    for idx in eye_points:
        lm = landmarks[idx]
        x, y = int(lm.x * image_w), int(lm.y * image_h)
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
        p.append((x, y))

    A = np.linalg.norm(np.array(p[1]) - np.array(p[5]))
    B = np.linalg.norm(np.array(p[2]) - np.array(p[4]))
    C = np.linalg.norm(np.array(p[0]) - np.array(p[3]))
    ear = (A + B) / (2.0 * C)
    return ear

def is_blinking(ear, threshold=0.2):
    return ear < threshold

def compute_concentration_score(gaze, head_pose, blink):
    score = 0.4 * gaze + 0.4 * head_pose + 0.2 * (0 if blink else 1)
    return round(score * 100, 2)

def bar(score, frame):
    """Draw concentration bar"""
    bar_x, bar_y = 30, 100
    bar_width, bar_height = 200, 30
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (50, 50, 50), -1)
    fill_width = int(score * bar_width / 100)
    color = (0, 255, 0) if score > 40 else (0, 100, 255)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + fill_width, bar_y + bar_height), color, -1)
    cv2.putText(frame, f"{score}%", (bar_x + bar_width + 10, bar_y + bar_height // 2 + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_stop_button(frame):
    """Draw a STOP button on the frame"""
    button_color = (0, 0, 255)
    button_text = "STOP"
    button_pos = (frame.shape[1] - 120, 20)
    button_size = (100, 50)
    cv2.rectangle(frame, button_pos, (button_pos[0] + button_size[0], button_pos[1] + button_size[1]), button_color, -1)
    cv2.putText(frame, button_text, (button_pos[0] + 10, button_pos[1] + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return button_pos, button_size

def is_button_clicked(event, x, y, flags, param):
    global stop_requested
    if event == cv2.EVENT_LBUTTONDOWN:
        bx, by, bw, bh = param
        if bx <= x <= bx + bw and by <= y <= by + bh:
            print("[INFO] Stop button clicked.")
            stop_requested = True

# --- Start camera ---
cap = cv2.VideoCapture(0)
stop_requested = False

cv2.namedWindow("Concentration Tracker")
cv2.setMouseCallback("Concentration Tracker", is_button_clicked, param=(0, 0, 0, 0))

while True:
    ret, frame = cap.read()
    if not ret or stop_requested:
        break

    # Draw UI background
    ui_bg = frame.copy()
    cv2.rectangle(ui_bg, (0, 0), (frame.shape[1], 150), (30, 30, 30), -1)
    cv2.addWeighted(ui_bg, 0.6, frame, 0.4, 0, frame)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_h, image_w, _ = frame.shape

    # Mediapipe face mesh
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                face_landmarks,
                mp_face_mesh.FACEMESH_CONTOURS
            )
            landmarks = face_landmarks.landmark
            left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, image_w, image_h)
            right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, image_w, image_h)
            avg_ear = (left_ear + right_ear) / 2
            blink = is_blinking(avg_ear)

            gaze_score = 1  # Placeholder (you can add actual gaze detection logic)
            head_score = 1  # Placeholder (you can add head pose detection logic)
            concentration = compute_concentration_score(gaze_score, head_score, blink)

            score_history.append(concentration)
            smooth_score = int(np.mean(score_history))
            bar(smooth_score, frame)

            cv2.putText(frame, f"Concentration: {smooth_score}%", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

            if blink:
                cv2.putText(frame, "BLINKING", (30, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 150, 255), 2)

    # YOLO object detection (phones and people)
    yolo_results = yolo.predict(source=frame, save=False, conf=0.3, verbose=False)
    for r in yolo_results:
        boxes = r.boxes
        for box in boxes:
            cls = int(box.cls[0])
            label = yolo.names[cls]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            if label in ["cell phone", "person"]:  # Detect phones and people
                color = (0, 0, 255) if label == "cell phone" else (255, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Draw stop button
    button_pos, button_size = draw_stop_button(frame)
    cv2.setMouseCallback("Concentration Tracker", is_button_clicked,
                         param=(button_pos[0], button_pos[1], button_size[0], button_size[1]))

    cv2.imshow("Concentration Tracker", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or stop_requested:
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Stop button clicked.


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque
from ultralytics import YOLO

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Load YOLOv8 (or replace with yolov10n.pt when ready)
yolo = YOLO("yolov8n.pt")  # Replace with "yolov10n.pt" if trained

# Eye landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

score_history = deque(maxlen=10)
distraction = 0

# Counters
phone_count = 0
background_person_count = 0

def eye_aspect_ratio(landmarks, eye_points, image_w, image_h):
    p = []
    for idx in eye_points:
        lm = landmarks[idx]
        x, y = int(lm.x * image_w), int(lm.y * image_h)
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
        p.append((x, y))
    A = np.linalg.norm(np.array(p[1]) - np.array(p[5]))
    B = np.linalg.norm(np.array(p[2]) - np.array(p[4]))
    C = np.linalg.norm(np.array(p[0]) - np.array(p[3]))
    ear = (A + B) / (2.0 * C)
    return ear

def is_blinking(ear, threshold=0.2):
    return ear < threshold

def compute_concentration_score(gaze, head_pose, blink):
    score = 0.4 * gaze + 0.4 * head_pose + 0.2 * (0 if blink else 1)
    return round(score * 100, 2)

def bar(score, frame):
    bar_x, bar_y = 30, 100
    bar_width, bar_height = 200, 30
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (50, 50, 50), -1)
    fill_width = int(score * bar_width / 100)
    color = (0, 255, 0) if score > 40 else (0, 100, 255)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + fill_width, bar_y + bar_height), color, -1)
    cv2.putText(frame, f"{score}%", (bar_x + bar_width + 10, bar_y + bar_height // 2 + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def draw_stop_button(frame):
    button_color = (0, 0, 255)
    button_text = "STOP"
    button_pos = (frame.shape[1] - 120, 20)
    button_size = (100, 50)
    cv2.rectangle(frame, button_pos, (button_pos[0] + button_size[0], button_pos[1] + button_size[1]), button_color, -1)
    cv2.putText(frame, button_text, (button_pos[0] + 10, button_pos[1] + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return button_pos, button_size

def is_button_clicked(event, x, y, flags, param):
    global stop_requested
    if event == cv2.EVENT_LBUTTONDOWN:
        bx, by, bw, bh = param
        if bx <= x <= bx + bw and by <= y <= by + bh:
            stop_requested = True

# Start camera
cap = cv2.VideoCapture(0)
stop_requested = False
cv2.namedWindow("Concentration Tracker")

# Mouse callback
cv2.setMouseCallback("Concentration Tracker", is_button_clicked, param=(0, 0, 0, 0))

while True:
    ret, frame = cap.read()
    if not ret or stop_requested:
        break

    # Draw UI background
    ui_bg = frame.copy()
    cv2.rectangle(ui_bg, (0, 0), (frame.shape[1], 150), (30, 30, 30), -1)
    cv2.addWeighted(ui_bg, 0.6, frame, 0.4, 0, frame)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_h, image_w, _ = frame.shape

    # Mediapipe face mesh
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)
            landmarks = face_landmarks.landmark
            left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, image_w, image_h)
            right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, image_w, image_h)
            avg_ear = (left_ear + right_ear) / 2
            blink = is_blinking(avg_ear)

            gaze_score = 1  # Placeholder
            head_score = 1  # Placeholder
            concentration = compute_concentration_score(gaze_score, head_score, blink)
            score_history.append(concentration)
            smooth_score = int(np.mean(score_history))
            bar(smooth_score, frame)
            cv2.putText(frame, f"Concentration: {smooth_score}%", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

            if blink:
                cv2.putText(frame, "BLINKING", (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 150, 255), 2)

    # YOLO object detection
    yolo_results = yolo.predict(source=frame, save=False, conf=0.4, verbose=False)
    for r in yolo_results:
        for box in r.boxes:
            cls = int(box.cls[0])
            label = yolo.names[cls]
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            if label == "cell phone":
                phone_count += 1
                color = (0, 0, 255)
            elif label == "person":
                background_person_count += 1
                color = (255, 255, 0)
            else:
                continue
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Draw STOP button
    button_pos, button_size = draw_stop_button(frame)
    cv2.setMouseCallback("Concentration Tracker", is_button_clicked,
                         param=(button_pos[0], button_pos[1], button_size[0], button_size[1]))

    cv2.imshow("Concentration Tracker", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or stop_requested:
        break

cap.release()
cv2.destroyAllWindows()

# Show counts after stopping
print("\n--- Detection Summary ---")
print(f"📱 Total phones detected: {phone_count}")
print(f"🧍‍♂️ Total background people detected: {background_person_count}")



--- Detection Summary ---
📱 Total phones detected: 165
🧍‍♂️ Total background people detected: 1855
